In [8]:
# json_inference_consumer.py

from kafka import KafkaConsumer, KafkaProducer
import os
import json
import uuid
from concurrent.futures import ThreadPoolExecutor


In [9]:

TOPIC_NAME = "avg-data"

KAFKA_SERVER = "kafka:29092"

NOTIFICATION_TOPIC = "NOTIFICATION"


In [10]:

consumer = KafkaConsumer(
    TOPIC_NAME,
    bootstrap_servers=[KAFKA_SERVER],
    auto_offset_reset='earliest',
    enable_auto_commit=False,
    # to deserialize kafka.producer.object into dict
    #value_deserializer=lambda m: json.loads(m.decode('utf-8')),
)


In [11]:
producer = KafkaProducer(
    bootstrap_servers = KAFKA_SERVER,
    api_version = (0, 11, 15)
)

In [12]:
def inferenceProcessFunction(data):
    # process steps with data in json format      
    
    json_data = json.loads(data)
    
    print("eventTime:" + str(json_data['eventTime']) + ' / ' + "nome:" + str(json_data['nome']) + ' / ' + "bpm:" + str(json_data['avgbpm']) + ' / ' + "temp:" + str(json_data['avgtemp']))
    
    if(json_data['avgbpm'] >= 120 or json_data['avgtemp'] >= 39):
        notification_data = data        
        producer.send(NOTIFICATION_TOPIC, notification_data)
        producer.flush()
        print('\nNotification Topic Sent!\n')
    

In [ ]:
for inf in consumer:
    print('processing line...')  
    print(inf.value)
    inf_data = inf.value
    inferenceProcessFunction(inf_data)

processing line...
b'{\n\t"eventTime": "2022-03-01 00:15:00",\n\t"nome": "antonio",\n\t"avgbpm": 130,\n\t"avgtemp": 42\n}'
eventTime:2022-03-01 00:15:00 / nome:antonio / bpm:130 / temp:42

Notification Topic Sent!

processing line...
b'{"eventTime":"2022-03-01 00:15:00","nome":"antonio","avgbpm":95.0,"avgtemp":41.0}'
eventTime:2022-03-01 00:15:00 / nome:antonio / bpm:95.0 / temp:41.0

Notification Topic Sent!

processing line...
b'{"eventTime":"2022-03-01 01:30:00","nome":"maria","avgbpm":63.0,"avgtemp":36.0}'
eventTime:2022-03-01 01:30:00 / nome:maria / bpm:63.0 / temp:36.0
processing line...
b'{"eventTime":"2022-03-01 01:45:00","nome":"antonio","avgbpm":95.0,"avgtemp":34.0}'
eventTime:2022-03-01 01:45:00 / nome:antonio / bpm:95.0 / temp:34.0
processing line...
b'{"eventTime":"2022-03-01 02:05:00","nome":"maria","avgbpm":93.0,"avgtemp":42.0}'
eventTime:2022-03-01 02:05:00 / nome:maria / bpm:93.0 / temp:42.0

Notification Topic Sent!

processing line...
b'{"eventTime":"2022-03-01 02:40